In [8]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from io import StringIO

In [9]:
service = Service(executable_path='../chromedriver.exe')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.get(f'https://www.investing.com/economic-calendar/')
time.sleep(2)

botton_cookies = driver.find_element(
    By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'
)
botton_cookies.click()
time.sleep(2)

for _ in range(30):
    botton_signup = driver.find_element(By.XPATH, '//*[@id="PromoteSignUpPopUp"]/div[2]/i')
    print('.', end='')
    if botton_signup.is_displayed():
        botton_signup.click()
        break
    else:
        time.sleep(2)


.....

In [4]:
botton_signup = driver.find_element(By.XPATH, '//*[@id="PromoteSignUpPopUp"]/div[2]/i')
botton_signup.click()

In [10]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(10)

In [15]:
table = driver.find_element(
    By.XPATH, '//*[@class="eCalNew eCalMainNew ecoCalCont"]'
)
dfs = pd.read_html(StringIO(table.get_attribute('innerHTML')))
df_raw = dfs[0]
if df_raw.columns[0] is not str: # Fix for ocasional multi-index
    df_raw.columns = [c[0] for c in df_raw.columns]
df_raw

,Time,Cur.,Imp.,Event,Actual,Forecast,Previous,Unnamed: 7,Unnamed: 8
0,"Friday, September 1, 2023","Friday, September 1, 2023","Friday, September 1, 2023","Friday, September 1, 2023","Friday, September 1, 2023","Friday, September 1, 2023","Friday, September 1, 2023","Friday, September 1, 2023","Friday, September 1, 2023"
1,All Day,NaN,Holiday,Hong Kong - Market Holiday,Hong Kong - Market Holiday,Hong Kong - Market Holiday,Hong Kong - Market Holiday,Hong Kong - Market Holiday,Hong Kong - Market Holiday
2,01:00,INR,NaN,S&P Global India Manufacturing PMI (Aug),58.6,57.5,57.7,NaN,NaN
3,02:00,GBP,NaN,Nationwide HPI (YoY),-5.3%,-3.9%,-3.8%,NaN,NaN
4,02:00,GBP,NaN,Nationwide HPI (MoM),-0.8%,-0.3%,-0.3%,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2160,19:30,JPY,NaN,National Core CPI (YoY) (Sep),NaN,2.7%,3.1%,NaN,NaN
2161,19:30,JPY,NaN,National CPI (YoY) (Sep),NaN,NaN,3.2%,NaN,NaN
2162,21:15,CNY,NaN,China Loan Prime Rate 5Y (Oct),NaN,4.20%,4.20%,NaN,NaN
2163,21:15,CNY,NaN,PBoC Loan Prime Rate,NaN,3.45%,3.45%,NaN,NaN


In [16]:
df_filtered = df_raw[~df_raw['Cur.'].isna()] # Exclude holidays
df_filtered.loc[:, 'Date'] = df_filtered['Imp.'].ffill() # Get dates for all rows
df_filtered = df_filtered[df_filtered['Imp.'].isna()] # Remove rows with dates

C:\Users\lukas\AppData\Local\Temp\ipykernel_15508\3107507473.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:, 'Date'] = df_filtered['Imp.'].ffill() # Get dates for all rows


In [17]:
flags = driver.find_elements(
    By.XPATH, '//*[@class="left flagCur noWrap"]/span'
)

countries = []
for flag in flags:
    countries.append(flag.get_attribute('title'))

df_filtered['Country'] = countries
df_filtered = df_filtered[~df_filtered['Actual'].isna()] # Get only indicators

In [18]:
df = df_filtered[['Time', 'Country', 'Cur.', 'Event',
                  'Actual', 'Forecast', 'Previous']]
df.index = pd.to_datetime(df_filtered['Date'])
df.index.name = None
df

,Time,Country,Cur.,Event,Actual,Forecast,Previous
2023-09-01,01:00,India,INR,S&P Global India Manufacturing PMI (Aug),58.6,57.5,57.7
2023-09-01,02:00,United Kingdom,GBP,Nationwide HPI (YoY),-5.3%,-3.9%,-3.8%
2023-09-01,02:00,United Kingdom,GBP,Nationwide HPI (MoM),-0.8%,-0.3%,-0.3%
2023-09-01,02:00,Russia,RUB,Russian S&P Global Manufacturing PMI (Aug),52.7,NaN,52.1
2023-09-01,02:30,Switzerland,CHF,CPI (MoM) (Aug),0.2%,0.2%,-0.1%
...,...,...,...,...,...,...,...
2023-10-19,10:00,United States,USD,Existing Home Sales (MoM) (Sep),-2.0%,NaN,-0.7%
2023-10-19,10:00,United States,USD,US Leading Index (MoM) (Sep),-0.7%,-0.4%,-0.5%
2023-10-19,10:30,United States,USD,Natural Gas Storage,97B,80B,84B
2023-10-19,11:30,United States,USD,4-Week Bill Auction,5.305%,NaN,5.325%


In [19]:
df[(df['Event'].str.contains('GDP'))]

,Time,Country,Cur.,Event,Actual,Forecast,Previous
2023-09-01,04:00,Italy,EUR,Italian GDP (YoY) (Q2),0.4%,0.6%,1.9%
2023-09-01,04:00,Italy,EUR,Italian GDP (QoQ) (Q2),-0.4%,-0.3%,0.6%
2023-09-01,08:00,Brazil,BRL,GDP (YoY) (Q2),3.4%,2.7%,4.0%
2023-09-01,08:00,Brazil,BRL,GDP (QoQ) (Q2),0.9%,0.3%,1.9%
2023-09-01,08:30,Canada,CAD,GDP Implicit Price (QoQ) (Q2),0.70%,1.50%,0.20%
...,...,...,...,...,...,...,...
2023-10-12,20:00,Singapore,SGD,GDP (YoY) (Q3),0.7%,0.4%,0.5%
2023-10-12,20:00,Singapore,SGD,GDP (QoQ) (Q3),1.0%,0.5%,0.1%
2023-10-17,22:00,China,CNY,GDP (YoY) (Q3),4.9%,4.4%,6.3%
2023-10-17,22:00,China,CNY,GDP (QoQ) (Q3),1.3%,1.0%,0.5%


In [20]:
df[(df['Country'] == 'Spain')]

,Time,Country,Cur.,Event,Actual,Forecast,Previous
2023-09-01,03:15,Spain,EUR,HCOB Spain Manufacturing PMI (Aug),46.5,48.8,47.8
2023-09-01,06:30,Spain,EUR,Car Registration (MoM) (Aug),-31.10%,NaN,-19.70%
2023-09-01,06:30,Spain,EUR,Car Registration (YoY) (Aug),7.80%,NaN,10.70%
2023-09-04,03:00,Spain,EUR,Spanish Unemployment Change,24.8K,-21.3K,-11.0K
2023-09-05,03:15,Spain,EUR,HCOB Spain Services PMI (Aug),49.3,51.5,52.8
2023-09-05,04:50,Spain,EUR,Spanish 12-Month Letras Auction,3.670%,NaN,3.664%
2023-09-05,04:50,Spain,EUR,Spanish 6-Month Letras Auction,3.654%,NaN,3.639%
2023-09-07,04:40,Spain,EUR,Spanish 10-Year Obligacion Auction,3.660%,NaN,3.605%
2023-09-07,04:40,Spain,EUR,Spanish 30-Year Obligacion Auction,4.190%,NaN,3.978%
2023-09-07,04:50,Spain,EUR,Spanish 3-Year Bonos Auction,3.304%,NaN,3.303%


In [21]:
df.groupby('Country')['Country'].count()

Country
Australia          78
Brazil             72
Canada             90
China              58
Euro Zone          90
France             77
Germany            85
Hong Kong          16
India              55
Italy              65
Japan             132
New Zealand        63
Russia             37
Singapore          24
South Africa       47
South Korea        37
Spain              52
Switzerland        33
United Kingdom    139
United States     563
Name: Country, dtype: int64